# 3. Inference example
- 모델 추론 예시입니다.
- 모델 생성 관련 huggingface transformer 공식 문서: https://huggingface.co/docs/transformers/main_classes/text_generation

In [1]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

/home/tako/anaconda3/envs/sk/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def generate_summary(test_samples, model, tokenizer, encoder_max_length, num_beams, min_length, max_length):
    inputs = tokenizer(
        test_samples["context"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, num_beams=num_beams, min_length = min_length, max_length = max_length,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [13]:
test_samples = {'context': '다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임위원회 등 의정활동을 위하여 2월 5일부터 2월 9일까지 5일간 본회의를 휴회하고자 하는데 의원 여러분 이의 있습니까? (“없습니다” 하는 의원 있음) 이의가 없으므로 가결되었음을 선포합니다. \
    이상으로 제247회 완주군의회 임시회 제1차 본회의를 마치겠습니다. 다음 제2차 본회의는 2월 10일 오전 10시에 개의하겠습니다. 의원 여러분 수고 많이 하셨습니다. 산회를 선포합니다.',\
    'summary': '2월 5일부터 2월 9일까지 5일간 본회의 휴회가 가결됨. 제2차 본회의는 2월 10일 오전 10시에 개의.'}

In [14]:
encoder_max_length = 512
num_beams = 16
min_length = 10
max_length = 50

model_pretrained = 'gogamza/kobart-summarization'

# Download model and tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_pretrained)

'''
Before training
'''
# Default pre-trained model is from https://github.com/seujung/KoBART-summarization 
# model = BartForConditionalGeneration.from_pretrained(model_pretrained)

'''
After training
'''
checkpoint_path = None # checkpoint path를 명시해주세요
model = BartForConditionalGeneration.from_pretrained(checkpoint_path)

predicted_summary = generate_summary(test_samples, model, tokenizer, encoder_max_length, num_beams, min_length, max_length)[1]
print(predicted_summary)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['2월 5일부터 2월 9일까지 5일간 휴회가 가결됨. 제247회 완주군의회 임시회 제1차 본회의는 2월 10일 오전 10시에 개의. 제2차 본회의는 2월 10일 오전 10시에 개의. 산회를 선포.']
